# Chest X-Ray Model 1

Date: 8/18/2024

Author: Sylas Chacko

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
from pathlib import Path


## Data Preprocessing Labels

In [ ]:
labels_df = pd.read_csv(r'C:\Users\sylas\OneDrive\Projects\medical_images\data\sample_labels.csv')
directory = r"C:\Users\sylas\OneDrive\Projects\medical_images\data\sample\sample\images"

# Have to adjust for _aug_i file name 

data = []

for i, row in labels_df.iterrows():
    file_name = row['Image Index']
#    f_names = row[['Image Index']+f'_aug_{i+1}']
    disease = row['Finding Labels']
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        for frame in labels_df:
            frame_dict = {'Disease' : disease, 'File_Name' : file_name}
    else:
        print('File not found: {}'.format(file_path))

if data:
    df = pd.DataFrame(frame_dict)
    df.to_csv('Image Labels')
    print('Image Labels csv created.')
else:
    print('No data processed. Check file paths and file availability.')
